In [10]:
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import os
from fpdf import FPDF

# Disable eager execution for TensorFlow 1.x compatibility
tf.compat.v1.disable_eager_execution()

# Set output directory for saving results
output_dir = "mnist_results"
os.makedirs(output_dir, exist_ok=True)

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.reshape(-1, 784) / 255.0, x_test.reshape(-1, 784) / 255.0
y_train_one_hot = np.eye(10)[y_train]
y_test_one_hot = np.eye(10)[y_test]

# Model hyperparameters
input_size = 784
hidden_size = 256  # Single Hidden Layer
output_size = 10
learning_rate = 0.001
batch_size = 100
epochs = 10  # Reduced for testing purpose
dropout_rate = 0.5
activations = {'sigmoid': tf.nn.sigmoid, 'relu': tf.nn.relu, 'tanh': tf.nn.tanh}

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

for act_name, activation in activations.items():
    print(f"\nTraining with Activation Function: {act_name}\n")

    X = tf.compat.v1.placeholder(tf.float32, [None, input_size])
    y = tf.compat.v1.placeholder(tf.float32, [None, output_size])
    keep_prob = tf.compat.v1.placeholder(tf.float32)

    # Initialize weights and biases
    weights = {
        'w1': tf.Variable(tf.random.truncated_normal([input_size, hidden_size], stddev=0.1)),
        'w2': tf.Variable(tf.random.truncated_normal([hidden_size, output_size], stddev=0.1))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([hidden_size])),
        'b2': tf.Variable(tf.zeros([output_size]))
    }

    # Neural network model
    layer1 = activation(tf.matmul(X, weights['w1']) + biases['b1'])
    layer1_drop = tf.nn.dropout(layer1, rate=1 - keep_prob)
    logits = tf.matmul(layer1_drop, weights['w2']) + biases['b2']

    # Loss, optimizer, accuracy
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(loss)
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    losses, accuracies = [], []
    start_time = time.time()

    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())

        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                batch_x, batch_y = x_train[i:i+batch_size], y_train_one_hot[i:i+batch_size]
                sess.run(optimizer, feed_dict={X: batch_x, y: batch_y, keep_prob: 1 - dropout_rate})

            train_loss, train_acc = sess.run([loss, accuracy], feed_dict={X: x_train, y: y_train_one_hot, keep_prob: 1.0})
            test_acc = sess.run(accuracy, feed_dict={X: x_test, y: y_test_one_hot, keep_prob: 1.0})
            losses.append(train_loss)
            accuracies.append(train_acc)
            print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, Test Acc: {test_acc*100:.2f}%")

        execution_time = time.time() - start_time
        final_test_preds = sess.run(tf.argmax(logits, 1), feed_dict={X: x_test, keep_prob: 1.0})

        # Save Loss Curve
        plt.figure()
        plt.plot(losses, label='Training Loss')
        plt.title(f'Loss Curve ({act_name})')
        plt.savefig(f"{output_dir}/loss_{act_name}.png")
        plt.close()

        # Save Confusion Matrix
        cm = confusion_matrix(y_test, final_test_preds)
        plt.figure(figsize=(10, 7))
        sns.heatmap(cm, annot=True, cmap='Blues', fmt='d')
        plt.title(f'Confusion Matrix ({act_name})')
        plt.savefig(f"{output_dir}/conf_matrix_{act_name}.png")
        plt.close()

        # Save Results to PDF
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, f"Activation Function: {act_name}")
        pdf.multi_cell(0, 10, f"Hidden Layer Size: {hidden_size}")
        pdf.multi_cell(0, 10, f"Final Test Accuracy: {test_acc * 100:.2f}%")
        pdf.multi_cell(0, 10, f"Execution Time: {execution_time:.2f} seconds")

        pdf.image(f"{output_dir}/loss_{act_name}.png", x=10, y=60, w=100)
        pdf.image(f"{output_dir}/conf_matrix_{act_name}.png", x=10, y=180, w=100)

# Save PDF Report
pdf_output_path = os.path.join(output_dir, "MNIST_Results_Report.pdf")
pdf.output(pdf_output_path)
print(f"\nPDF Report saved successfully at: {pdf_output_path}")



Training with Activation Function: sigmoid

Epoch 1, Loss: 0.3030, Train Acc: 91.22%, Test Acc: 91.78%
Epoch 2, Loss: 0.2298, Train Acc: 93.35%, Test Acc: 93.34%
Epoch 3, Loss: 0.1880, Train Acc: 94.55%, Test Acc: 94.41%
Epoch 4, Loss: 0.1578, Train Acc: 95.39%, Test Acc: 95.10%
Epoch 5, Loss: 0.1351, Train Acc: 96.05%, Test Acc: 95.76%
Epoch 6, Loss: 0.1176, Train Acc: 96.62%, Test Acc: 96.18%
Epoch 7, Loss: 0.1058, Train Acc: 96.94%, Test Acc: 96.48%
Epoch 8, Loss: 0.0934, Train Acc: 97.33%, Test Acc: 96.79%
Epoch 9, Loss: 0.0847, Train Acc: 97.59%, Test Acc: 96.90%
Epoch 10, Loss: 0.0768, Train Acc: 97.80%, Test Acc: 97.12%

Training with Activation Function: relu

Epoch 1, Loss: 0.1755, Train Acc: 94.88%, Test Acc: 94.79%
Epoch 2, Loss: 0.1181, Train Acc: 96.51%, Test Acc: 96.09%
Epoch 3, Loss: 0.0914, Train Acc: 97.31%, Test Acc: 96.62%
Epoch 4, Loss: 0.0729, Train Acc: 97.86%, Test Acc: 97.10%
Epoch 5, Loss: 0.0611, Train Acc: 98.20%, Test Acc: 97.38%
Epoch 6, Loss: 0.0542, Trai